In [1]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import pickle
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

from torchvision.models import resnet101,mobilenet_v2
import sys
import timm
sys.path.insert(0, 'timm==0.4.5')
#import timm #gets version 2
print(f"Timm: {timm.__version__}")
print(f"Torch: {torch.__version__}")

Timm: 0.4.5
Torch: 1.9.0+cu102


In [2]:
IMG_SIZE = 224
use_cuda = torch.cuda.is_available()
batch_size=2

test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

test_dir = r'C:\Users\HP\FER_SL\sl_data\test_files_cropped'

kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}

test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transforms)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs) 

print(len(test_dataset))

507


In [3]:
new_model = torch.load(r'C:\Users\HP\FER_SL\models\affectnet_emotions\enet_b0_7.pt')
new_model.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0

In [4]:
from facial_analysis import FacialImageProcessing
imgProcessing=FacialImageProcessing(False)

In [5]:
class_to_idx = {'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
idx_to_class = {0: 'anger', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

In [6]:
device = 'cuda'
y_val,y_scores_val=[],[]
#new_model.eval()
for class_name in tqdm(os.listdir(test_dir)):
    if class_name in class_to_idx:
        class_dir=os.path.join(test_dir,class_name)
        y=class_to_idx[class_name]
        for img_name in os.listdir(class_dir):
            filepath=os.path.join(class_dir,img_name)
            img = Image.open(filepath)
            img_tensor = test_transforms(img)
            img_tensor.unsqueeze_(0)
            scores = new_model(img_tensor.to(device))
            scores=scores[0].data.cpu().numpy()
            #print(scores.shape)
            y_scores_val.append(scores)
            y_val.append(y)

y_scores_val=np.array(y_scores_val)
y_val=np.array(y_val)
print(y_scores_val.shape,y_val.shape)

  0%|          | 0/7 [00:00<?, ?it/s]

(507, 7) (507,)


In [7]:
y_pred=np.argmax(y_scores_val,axis=1)
acc=100.0*(y_val==y_pred).sum()/len(y_val)
print(acc)

targets = ['anger','disgust','fear','happy','neutral','sad','surprise']

for i in range(y_scores_val.shape[1]):
    _val_acc=(y_pred[y_val==i]==i).sum()/(y_val==i).sum()
    print('%s acc: %f' %(idx_to_class[i],100*_val_acc))

53.451676528599606
anger acc: 50.757576
disgust acc: 67.741935
fear acc: 36.842105
happy acc: 47.368421
neutral acc: 47.619048
sad acc: 18.644068
surprise acc: 73.053892


In [8]:
from sklearn.metrics import classification_report

target_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']
print(classification_report(y_val, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Anger       0.83      0.51      0.63       132
     Disgust       0.20      0.68      0.31        31
        Fear       0.31      0.37      0.34        57
       Happy       0.33      0.47      0.39        19
     Neutral       0.41      0.48      0.44        42
     Sadness       0.61      0.19      0.29        59
    Surprise       0.77      0.73      0.75       167

    accuracy                           0.53       507
   macro avg       0.49      0.49      0.45       507
weighted avg       0.63      0.53      0.55       507



# Accuracy

In [9]:
from sklearn.metrics import accuracy_score

acc=100.0*(y_val==y_pred).sum()/len(y_val)
print('Accuracy:',acc)

Accuracy: 53.451676528599606


In [10]:
# Weighted accuracy

from sklearn.metrics import f1_score
w_acc = f1_score(y_val, y_pred, average='weighted')
print(w_acc)

0.5514786661284384


In [11]:
from sklearn.metrics import classification_report
clas_report = classification_report(y_val, y_pred, labels=None, target_names=None, sample_weight=None)
print(clas_report)

              precision    recall  f1-score   support

           0       0.83      0.51      0.63       132
           1       0.20      0.68      0.31        31
           2       0.31      0.37      0.34        57
           3       0.33      0.47      0.39        19
           4       0.41      0.48      0.44        42
           5       0.61      0.19      0.29        59
           6       0.77      0.73      0.75       167

    accuracy                           0.53       507
   macro avg       0.49      0.49      0.45       507
weighted avg       0.63      0.53      0.55       507



# Confusion Matrix

In [12]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_val, y_pred, normalize='true')
print(idx_to_class.values())
print(cm)

dict_values(['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'])
[[0.50757576 0.28787879 0.06818182 0.         0.0530303  0.02272727
  0.06060606]
 [0.06451613 0.67741935 0.09677419 0.09677419 0.03225806 0.
  0.03225806]
 [0.         0.31578947 0.36842105 0.01754386 0.01754386 0.05263158
  0.22807018]
 [0.         0.05263158 0.         0.47368421 0.21052632 0.
  0.26315789]
 [0.         0.         0.11904762 0.30952381 0.47619048 0.02380952
  0.07142857]
 [0.16949153 0.28813559 0.05084746 0.01694915 0.16949153 0.18644068
  0.11864407]
 [0.01197605 0.06586826 0.15568862 0.         0.03592814 0.
  0.73053892]]


# Sensitivity

In [21]:
# Sensitivity per class

from sklearn.metrics import recall_score
sensitivity_per_class = recall_score(y_val,y_pred,average=None)
print(sensitivity_per_class)

[0.49635036 0.67741935 0.36842105 0.47368421 0.47619048 0.18644068
 0.73053892]


In [25]:
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity )


Sensitivity :  0.6238532110091743
